# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [5]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
!pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.0+cpu (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2, 1.7.1, 1.7.1+cpu, 1.7.1+cu101, 1.7.1+cu110, 1.7.1+cu92, 1.7.1+rocm3.7, 1.7.1+rocm3.8, 1.8.0, 1.8.0+cpu, 1.8.0+cu101, 1.8.0+cu111, 1.8.0+rocm3.10, 1.8.0+rocm4.0.1, 1.8.1, 1.8.1+cpu, 1.8.1+cu101, 1.8.1+cu102, 1.8.1+cu111, 1.8.1+rocm3.10, 1.8.1+rocm4.0.1, 1.9.0, 1.9.0+cpu, 1.9.0+cu102, 1.9.0+cu111, 1.9.0+rocm4.0.1, 1.9.0+rocm4.1, 1.9.0+rocm4.2)
ERROR: No matching distribution found for torch==1.7.0+cpu


In [6]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [7]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [8]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

  0%|          | 0/55628 [00:00<?, ?it/s]

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [9]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [10]:
your_name = 'Anirban' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [11]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [12]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
27,55,female,36.0525,2,no,northwest,12882.063862
752,64,male,41.6955,0,no,northwest,14921.062747
1258,55,male,41.4865,3,no,northwest,31566.759578
384,44,male,24.3485,2,no,northeast,8717.662432
406,33,female,26.7410,0,no,southeast,4394.352795


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [13]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [14]:
num_cols = len(dataframe.columns)
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [15]:
input_cols = list(dataframe.columns[:-1])
print(input_cols)

['age', 'sex', 'bmi', 'children', 'smoker', 'region']


**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [16]:
categorical_cols = dataframe.select_dtypes(include=['object']).columns
categorical_cols

Index(['sex', 'smoker', 'region'], dtype='object')

**Q: What are the column titles of output/target variable(s)?**

In [17]:
output_cols = [dataframe.columns[-1]]
output_cols

['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [18]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [19]:
!pip install jovian --upgrade -q

In [20]:
import jovian

In [21]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "anirbansaha/02-insurance-linear-regression" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/anirbansaha/02-insurance-linear-regression


'https://jovian.ai/anirbansaha/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [22]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [23]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[55.    ,  0.    , 36.0525,  2.    ,  0.    ,  1.    ],
        [64.    ,  1.    , 41.6955,  0.    ,  0.    ,  1.    ],
        [55.    ,  1.    , 41.4865,  3.    ,  0.    ,  1.    ],
        ...,
        [56.    ,  1.    , 28.5285,  0.    ,  0.    ,  0.    ],
        [18.    ,  0.    , 33.3355,  0.    ,  0.    ,  0.    ],
        [43.    ,  1.    , 28.633 ,  0.    ,  0.    ,  0.    ]]),
 array([[12882.0638625],
        [14921.0627475],
        [31566.7595775],
        ...,
        [11723.6885325],
        [ 2313.9227475],
        [ 7179.237135 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [24]:
inputs = torch.tensor(inputs_array).type(torch.float32)
targets = torch.tensor(targets_array).type(torch.float32)

In [25]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [26]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [27]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size,val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [28]:
batch_size = 5

In [29]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [30]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[56.0000,  0.0000, 41.2610,  2.0000,  0.0000,  2.0000],
        [42.0000,  0.0000, 27.8300,  1.0000,  0.0000,  3.0000],
        [40.0000,  0.0000, 25.7070,  3.0000,  0.0000,  0.0000],
        [36.0000,  0.0000, 28.4240,  0.0000,  0.0000,  1.0000],
        [40.0000,  0.0000, 31.5590,  3.0000,  0.0000,  1.0000]])
targets: tensor([[12878.7822],
        [ 7397.7739],
        [ 8664.8984],
        [ 5529.6841],
        [ 8462.6631]])


Let's save our work by committing to Jovian.

In [31]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "anirbansaha/02-insurance-linear-regression" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/anirbansaha/02-insurance-linear-regression


'https://jovian.ai/anirbansaha/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [32]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [33]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                   # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.smooth_l1_loss(out, targets)                           # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.smooth_l1_loss(out, targets)                            # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [34]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3518, -0.3849,  0.1018, -0.0043, -0.0490,  0.3688]],
        requires_grad=True),
 Parameter containing:
 tensor([0.3809], requires_grad=True)]

One final commit before we train the model.

In [36]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "anirbansaha/02-insurance-linear-regression" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/anirbansaha/02-insurance-linear-regression


'https://jovian.ai/anirbansaha/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [37]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [38]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 13624.14453125}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [39]:
epochs = 50
lr = 0.5
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7055.9902
Epoch [40], val_loss: 7925.2559
Epoch [50], val_loss: 7297.3657


In [40]:
epochs = 100
lr = 0.3
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6912.3691
Epoch [40], val_loss: 6831.8750
Epoch [60], val_loss: 6945.4121
Epoch [80], val_loss: 6627.9761
Epoch [100], val_loss: 6831.1226


In [41]:
epochs = 500
lr = 0.1
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6583.5459
Epoch [40], val_loss: 6532.5791
Epoch [60], val_loss: 6582.5425
Epoch [80], val_loss: 6510.5532
Epoch [100], val_loss: 6473.3965
Epoch [120], val_loss: 6441.8921
Epoch [140], val_loss: 6460.7827
Epoch [160], val_loss: 6406.8428
Epoch [180], val_loss: 6449.7983
Epoch [200], val_loss: 6361.0405
Epoch [220], val_loss: 6389.7246
Epoch [240], val_loss: 6316.5669
Epoch [260], val_loss: 6299.7334
Epoch [280], val_loss: 6271.8579
Epoch [300], val_loss: 6271.3110
Epoch [320], val_loss: 6237.5098
Epoch [340], val_loss: 6337.3027
Epoch [360], val_loss: 6223.1631
Epoch [380], val_loss: 6167.1060
Epoch [400], val_loss: 6174.4160
Epoch [420], val_loss: 6194.1616
Epoch [440], val_loss: 6162.8750
Epoch [460], val_loss: 6095.1523
Epoch [480], val_loss: 6110.1104
Epoch [500], val_loss: 6073.0312


In [42]:
epochs = 500
lr = 0.01
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6039.3867
Epoch [40], val_loss: 6036.8550
Epoch [60], val_loss: 6033.8345
Epoch [80], val_loss: 6032.0508
Epoch [100], val_loss: 6031.1040
Epoch [120], val_loss: 6029.0918
Epoch [140], val_loss: 6033.2583
Epoch [160], val_loss: 6022.8584
Epoch [180], val_loss: 6021.7485
Epoch [200], val_loss: 6018.6348
Epoch [220], val_loss: 6017.7930
Epoch [240], val_loss: 6016.3535
Epoch [260], val_loss: 6021.7666
Epoch [280], val_loss: 6012.7925
Epoch [300], val_loss: 6011.0210
Epoch [320], val_loss: 6008.9023
Epoch [340], val_loss: 6006.8306
Epoch [360], val_loss: 6003.4004
Epoch [380], val_loss: 6004.9854
Epoch [400], val_loss: 6005.2695
Epoch [420], val_loss: 5998.8555
Epoch [440], val_loss: 5995.8047
Epoch [460], val_loss: 5995.4033
Epoch [480], val_loss: 5991.6670
Epoch [500], val_loss: 5989.2173


In [43]:
epochs = 500
lr = 0.0001
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5989.1758
Epoch [40], val_loss: 5989.2383
Epoch [60], val_loss: 5989.3223
Epoch [80], val_loss: 5989.3677
Epoch [100], val_loss: 5989.4126
Epoch [120], val_loss: 5989.4766
Epoch [140], val_loss: 5989.4990
Epoch [160], val_loss: 5989.5571
Epoch [180], val_loss: 5989.5908
Epoch [200], val_loss: 5989.6313
Epoch [220], val_loss: 5989.6758
Epoch [240], val_loss: 5989.7129
Epoch [260], val_loss: 5989.7456
Epoch [280], val_loss: 5989.8062
Epoch [300], val_loss: 5989.8486
Epoch [320], val_loss: 5989.9048
Epoch [340], val_loss: 5989.9355
Epoch [360], val_loss: 5989.9648
Epoch [380], val_loss: 5989.9761
Epoch [400], val_loss: 5990.0205
Epoch [420], val_loss: 5990.0522
Epoch [440], val_loss: 5990.0957
Epoch [460], val_loss: 5990.1313
Epoch [480], val_loss: 5990.1689
Epoch [500], val_loss: 5990.2114


**Q: What is the final validation loss of your model?**

In [44]:
val_loss = evaluate(model, val_loader)
val_loss

{'val_loss': 5990.21142578125}

Let's log the final validation loss to Jovian and commit the notebook

In [52]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [ ]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [47]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [48]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([58.0000,  1.0000, 31.4545,  0.0000,  0.0000,  1.0000])
Target: tensor([12322.6729])
Prediction: tensor([12364.8750])


In [49]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([26.0000,  1.0000, 37.3065,  1.0000,  0.0000,  1.0000])
Target: tensor([3457.1562])
Prediction: tensor([3802.7466])


In [50]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([18.0000,  0.0000, 43.0760,  0.0000,  0.0000,  2.0000])
Target: tensor([1714.6967])
Prediction: tensor([1255.8888])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [ ]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes